In [27]:
import pprint
import numpy as np
import pandas as pd
import power_grid_model
from power_grid_model import PowerGridModel, CalculationMethod, CalculationType
from power_grid_model import LoadGenType

from power_grid_model.utils import json_deserialize, json_deserialize_from_file, json_serialize, json_serialize_to_file
import json


# Read the data

In [23]:
with open("data/serialized_input.json", "r") as f:
    ser_input = f.read()

#pprint.pprint(json.loads(ser_input))

pgm_input = json_deserialize(ser_input)

display(pd.DataFrame(pgm_input["node"]))
display(pd.DataFrame(pgm_input["line"]))
display(pd.DataFrame(pgm_input["sym_load"]))
display(pd.DataFrame(pgm_input["source"]))

#print(pd.DataFrame(json.loads(ser_input))) # Doesnot read the whole dataset, concantenates and cannot open at all




,id,u_rated
0,1,10500.0
1,2,10500.0
2,3,10500.0


,id,from_node,to_node,from_status,to_status,r1,x1,c1,tan1,r0,x0,c0,tan0,i_n
0,4,1,2,1,1,0.11,0.12,0.00004,0.10,NaN,NaN,NaN,NaN,500.0
1,5,2,3,1,1,0.15,0.16,0.00005,0.12,NaN,NaN,NaN,NaN,550.0


,id,node,status,type,p_specified,q_specified
0,7,2,1,0,1010000.0,210000.0
1,8,3,1,0,1020000.0,220000.0


,id,node,status,u_ref,u_ref_angle,sk,rx_ratio,z01_ratio
0,15,1,1,1.03,NaN,1.000000e+20,NaN,NaN
1,16,1,1,1.04,NaN,NaN,NaN,NaN
2,17,1,1,1.03,NaN,1.000000e+10,0.2,NaN


In [32]:
model = PowerGridModel(pgm_input)   

pgm_output = model.calculate_power_flow(
    symmetric=True,
    error_tolerance=1e-8,
    max_iterations=20,
    calculation_method=CalculationMethod.newton_raphson)

In [33]:
print("------node result------")
display(pd.DataFrame(pgm_output["node"]))
print("------line result------")
display(pd.DataFrame(pgm_output["line"]))
print("------Sym_loads result------")
display(pd.DataFrame(pgm_output["sym_load"]))
print("------source result------")
display(pd.DataFrame(pgm_output["source"]))

------node result------


,id,energized,u_pu,u,u_angle,p,q
0,1,1,1.030000,10815.000000,-2.527860e-14,2.408998e+06,-2.863495e+06
1,2,1,1.029997,10814.968183,-4.398000e-03,-1.010000e+06,-2.100000e+05
2,3,1,1.029484,10809.581008,-6.839956e-03,-1.020000e+06,-2.200000e+05


------line result------


,id,energized,loading,p_from,q_from,i_from,s_from,p_to,q_to,i_to,s_to
0,4,1,0.399532,2.408998e+06,-2.863495e+06,199.765955,3.742042e+06,-2.252626e+06,1.403929e+06,141.698433,2.654306e+06
1,5,1,0.197705,1.242626e+06,-1.613929e+06,108.737609,2.036881e+06,-1.020000e+06,-2.200000e+05,55.731992,1.043456e+06


------Sym_loads result------


,id,energized,p,q,i,s,pf
0,7,1,1010000.0,210000.0,55.071354,1.031601e+06,0.979061
1,8,1,1020000.0,220000.0,55.731992,1.043456e+06,0.977521


------source result------


,id,energized,p,q,i,s,pf
0,15,1,-7.840728e+06,-1.053591e+08,5.640066e+03,1.056504e+08,-0.074214
1,16,1,1.024888e+07,1.024888e+08,5.498574e+03,1.030000e+08,0.099504
2,17,1,-1.808335e-03,-1.040914e-02,5.640066e-07,1.056504e-02,-0.171162
